In [15]:
from queue import Queue
from unidecode import unidecode
import networkx as nx
import spotipy #https://spotipy.readthedocs.io/en/latest/
from spotipy.oauth2 import SpotifyClientCredentials
import statistics 

In [16]:
# Charles profile link:
# https://open.spotify.com/user/12167056236?si=BfbG8v9wRsOTnilpjP9ibg
# spotify:user:12167056236:playlist:1ahkvZq6vSkYCmqHiPpLJn
charles_id = '12167056236'

# Jazz playlist link:
# https://open.spotify.com/user/12167056236/playlist/2ek4gj4PDBF4aspnYWfBmC?si=eq_YK6V8SdKiuhu1zj5VPA
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC
jazz_id = '2ek4gj4PDBF4aspnYWfBmC'

# Bossa nova playlist link:
# https://open.spotify.com/user/12167056236/playlist/1ahkvZq6vSkYCmqHiPpLJn?si=DQT4XTRYSwWagYejOs34Ew
bossanova_id = '1ahkvZq6vSkYCmqHiPpLJn'

In [17]:
# client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC

In [18]:
# playlists = sp.user_playlists('spotify')
# tracks = user_playlist_tracks(user = 12167056236, playlist_id=2ek4gj4PDBF4aspnYWfBmC?si=j-MSydgGR1O3KgIUea6Yzw)

In [19]:
client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# def calc_feat_dict(results):
#     artists_musics = {}
#     for i, item in enumerate(results['items']):
#         track = item['track']
#         artists_musics[track['artists'][0]['name']] = (track['name'].split(" - ")[0])
#         #print(track['id'])
#         #print("%d: %s - %s" % (i, track['artists'][0]['name'], track['name'].split(" - ")[0]))
#     return artists_musics

def calc_tuple(results):
    artists_musics = []
    for i, item in enumerate(results['items']):
        track = item['track']
        artists_musics.append((track['artists'][0]['id'], track['artists'][0]['name'], track['id'], track['name']))
        #print((track['artists'][0]['name'], (track['name'].split(" - ")[0])))
    return artists_musics

In [34]:
art_mus = {}

results = sp.user_playlist(user = charles_id, playlist_id = jazz_id, fields = "tracks, next")
tracks = results['tracks']

while tracks['next']: #passa pelos 10 conjuntos de tracks
    tracks = sp.next(tracks)
    data = calc_tuple(tracks)    
    
    #inicializa o dicio com o primeiro artista deste cojunto
    for i in range(len(data)): #rodará 100 vezes
        if data[i][0] not in art_mus.keys(): #trocou artista
            art_mus[data[i][0]] = [data[i][2]]
        else: #mesmo artista
            art_mus[data[i][0]].append(data[i][2])

artistas = 119
#print(art_mus)

In [33]:
id_art = {}

results = sp.user_playlist(user = charles_id, playlist_id = jazz_id, fields = "tracks, next")
tracks = results['tracks']

while tracks['next']: #passa pelos 10 conjuntos de tracks
    tracks = sp.next(tracks)
    data = calc_tuple(tracks)    
    
    #inicializa o dicio com o primeiro artista deste cojunto
    for i in range(len(data)): #rodará 100 vezes
        id_art[data[i][0]] = [data[i][1]]

In [25]:
# https://developer.spotify.com/documentation/web-api/reference/object-model/#audio-features-object
art_feats = {}

# loop para obter as 'features' das músicas e adicionar à um novo dicio
for artist, songs in art_mus.items():
    if (len(songs) > 49): # limitação na api do spotify de 'track features' é de 50 músicas
        for song in songs:
            if artist not in art_feats.keys():
                art_feats[artist] = [sp.audio_features(song)[0]]
            else:
                art_feats[artist].append(sp.audio_features(song)[0])
    else:
        art_feats[artist] = sp.audio_features(songs)
        feats = sp.audio_features(songs)
art_feats

{'03YhcM6fxypfwckPCQV8pQ': [{'acousticness': 0.542,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7lJuSy7GdlCiRf8bnaxzCO',
   'danceability': 0.545,
   'duration_ms': 285040,
   'energy': 0.389,
   'id': '7lJuSy7GdlCiRf8bnaxzCO',
   'instrumentalness': 0.136,
   'key': 2,
   'liveness': 0.243,
   'loudness': -15.116,
   'mode': 0,
   'speechiness': 0.029,
   'tempo': 125.41,
   'time_signature': 4,
   'track_href': 'https://api.spotify.com/v1/tracks/7lJuSy7GdlCiRf8bnaxzCO',
   'type': 'audio_features',
   'uri': 'spotify:track:7lJuSy7GdlCiRf8bnaxzCO',
   'valence': 0.792},
  {'acousticness': 0.907,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1O5yX7WJDvhMk7BSIAC6vP',
   'danceability': 0.676,
   'duration_ms': 229173,
   'energy': 0.202,
   'id': '1O5yX7WJDvhMk7BSIAC6vP',
   'instrumentalness': 0.885,
   'key': 7,
   'liveness': 0.114,
   'loudness': -22.411,
   'mode': 0,
   'speechiness': 0.0383,
   'tempo': 146.774,
   'time_signature': 4,
   'track_

In [29]:
art_mus_feats = {}
#  loop para obter as médias das 'features' das músicas de um artista
for artist, feats in art_feats.items():
#     print(type(feats))
#     print(artist)
#     for i in feats:
#         print(i)
    art_mus_feats[artist] = [
        {
            'danceability': statistics.mean([x.get('danceability') for x in feats]),
            'energy': statistics.mean([x.get('energy') for x in feats]),
            'key': statistics.mean([x.get('key') for x in feats]),
            'loudness': statistics.mean([x.get('loudness') for x in feats]),
            'mode': statistics.mean([x.get('mode') for x in feats]),
            'speechiness': statistics.mean([x.get('speechiness') for x in feats]),
            'acousticness': statistics.mean([x.get('acousticness') for x in feats]),
            'instrumentalness': statistics.mean([x.get('instrumentalness') for x in feats]),
            'liveness': statistics.mean([x.get('liveness') for x in feats]),
            'valence': statistics.mean([x.get('valence') for x in feats]),
            'tempo': statistics.mean([x.get('tempo') for x in feats]),
#             '': statistics.mean([x.get('key') for x in feats]),
#             '': statistics.mean([x.get('key') for x in feats]),
        }
    ]

art_mus_feats['03YhcM6fxypfwckPCQV8pQ']

[{'acousticness': 0.7245,
  'danceability': 0.6105,
  'energy': 0.2955,
  'instrumentalness': 0.5105,
  'key': 4.5,
  'liveness': 0.1785,
  'loudness': -18.7635,
  'mode': 0,
  'speechiness': 0.03365,
  'tempo': 136.09199999999998,
  'valence': 0.7495}]

In [ ]:
GRAPH_NAME = 'spotify'
class SpotifyWrapper:
    def __init__(self):
        credentials = SpotifyClientCredentials(
            client_id='d9a2223d179b4d9b845953cdc3b1f49b',
            client_secret='0b2e5ce4a3d345b6a31793c13141b3aa',
        )
 
        self.spotify = Spotify(client_credentials_manager=credentials)
 
 
    def filter(self, artist):
        return {key: artist[key] for key in ('name', 'popularity')}
 
 
    def artist(self, n):
        artist = self.spotify.artist(n)
 
        return self.filter(artist)
 
 
    def related_artists(self, n):
        related = self.spotify.artist_related_artists(n)
 
        for artist in related['artists']:
            yield artist['id'], self.filter(artist)
 
 
def add_node(g, n, artist):
    g.add_node(n)
 
    for key in artist:
        g.nodes[n][key] = artist[key]


In [ ]:
#SEED_UID_1 = '6XpaIBNiVzIetEPCWDvAFP' # Whitney Houston
#SEED_UID_2 = '4NJhFmfw43RLBLjQvxDuRS' # Wolfgang Amadeus Mozart

q = Queue()
g = nx.DiGraph()
w = SpotifyWrapper()

q.put(SEED_UID_1)
add_node(g, SEED_UID_1, w.artist(SEED_UID_1))

q.put(SEED_UID_2)
add_node(g, SEED_UID_2, w.artist(SEED_UID_2))

while not q.empty():
    print(g.number_of_nodes(), g.number_of_edges())

    n = q.get()

    for m, artist in w.related_artists(n):
        if not g.has_node(m):
            q.put(m)
            add_node(g, m, artist)

        if not g.has_edge(n, m):
            g.add_edge(n, m)

    n = SEED_UID_1
    m = SEED_UID_2

    if nx.has_path(g, n, m) and nx.has_path(g, m, n):
        print('connected')
        break

nodes = list(g.nodes)

with open(GRAPH_NAME + '.gml', 'w') as file:
    file.write('graph [\n')
    file.write('  directed 1\n')

    for i, n in enumerate(nodes):
        name = unidecode(g.nodes[n]['name']).replace('"', "'")
        popularity = g.nodes[n]['popularity']

        file.write('  node [\n')
        file.write('    id {}\n'.format(i))
        file.write('    label "{}"\n'.format(name))
        file.write('    popularity {}\n'.format(popularity))
        file.write('  ]\n')

    for n, m in g.edges:
        file.write('  edge [\n')
        file.write('    source {}\n'.format(nodes.index(n)))
        file.write('    target {}\n'.format(nodes.index(m)))
        file.write('  ]\n')

    file.write(']\n')